# Long Sentence Resolution - Run Solution Recipe (v1)

# Documentation

## Connect Google Drive

*   Run the cell "Connect Google Drive" and on the popup window click  "Connect to Google Drive".

*   In the new page opened, choose an account (your account) and click Allow.

## Configuration parameters

* **Shortening model checkpoint**: The shortening model checkpoint to load (default: shortening_model_2022-10-21-21-08-16/checkpoint-61551).

* **Splitting model checkpoint**: The splitting model checkpoint to load (default: splitting_model_2022-10-27-15-34-55/checkpoint-135954).

* **Target maximum #words**: The maximum number of words a sentence should have to be considered a valid target sentence (default: 15).

* **Shortening/Splitting #words threshold**: When the #words in a sentence is less than the threshold, shortening is applied to it. Otherwise, splitting is applied (default: 20).

* **Long sentences to resolve**: The long sentences to be resolved. Each line in the text area should have only one sentence.

## Models

Model ID | Model best checkpoint | Model base checkpoint | Dataset ID (trained on) | Model task | Model architecture | Dataset sources | Tasks of dataset sources
:- | :- | :- | :- | :- | :- | :- | :-
shortening_model_2022-10-24-08-53-31 | checkpoint-90000 | facebook/bart-large | shortening_dataset_2022-10-20-10-12-03 | Sentence Compression | BART | Google sentence compression dataset | Sentence Compression
shortening_model_2022-10-21-08-59-07 | checkpoint-2650 | facebook/bart-large | shortening_dataset_2022-10-20-10-26-09 | Sentence Compression | BART | QQP, MRPC, PAWS | Paraphrasing
shortening_model_2022-10-21-21-08-16 | checkpoint-61551 | facebook/bart-large | shortening_dataset_2022-10-20-11-03-23 | Sentence Compression | BART | Google sentence compression dataset, QQP, MRPC, PAWS | Sentence Compression, Paraphrasing
shortening_model_2022-10-17-11-14-41 | checkpoint-300000 | facebook/bart-large | shortening_dataset_2022-10-17-08-29-25 | Sentence Compression | BART | Gigaword, DUC 2004, Google sentence compression dataset, QQP, MRPC, PAWS | Summarization, Sentence Compression, Paraphrasing
splitting_model_2022-10-26-09-29-01 | checkpoint-55230 | facebook/bart-large | splitting_dataset_2022-10-20-11-08-22 | Split and Rephrase | BART | WikiSplit | Split and Rephrase
splitting_model_2022-10-27-06-27-03 | checkpoint-53112 | facebook/bart-large | splitting_dataset_2022-10-20-11-12-34 | Split and Rephrase | BART | BiSECT | Split and Rephrase
splitting_model_2022-10-27-15-34-55 | checkpoint-135954 | facebook/bart-large | splitting_dataset_2022-10-20-11-14-41 | Split and Rephrase | BART | WikiSplit, BiSECT | Split and Rephrase
splitting_model_2022-10-27-10-15-31 | N/A | N/A | splitting_dataset_2022-10-20-11-14-41 | Split | Baseline | WikiSplit, BiSECT | Split and Rephrase

## Solution recipe flowchart (for a single sentence)

image.png

# Installations

In [ ]:
!pip install transformers==4.23.1 \
             spacy==3.4.1

# Imports

In [ ]:
import os, torch, glob

import pandas as pd

import numpy as np

import ipywidgets as widgets

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from pathlib import Path

from typing import List, Union

from google.colab import drive

from spacy.lang.en import English

from spacy.tokens import Doc, Token, Span

# Functions / Classes

In [ ]:
class DocumentAnalysis:
  def __init__ (self):
    self._linguistics_model = English()

    self._linguistics_model.add_pipe("sentencizer")

  def extract_n_words_from_documents(self, documents: Union[List[str], np.ndarray, pd.Series]) -> np.ndarray:
    return np.array(list(map(self._extract_n_words_from_document, self._linguistics_model.pipe(documents, batch_size=50))))

  def extract_sentence_n_words_from_documents(self, documents: Union[List[str], np.ndarray, pd.Series]) -> pd.Series:
    series_index = None if type(documents) != pd.Series else documents.index

    return pd.Series(list(map(self._extract_sentence_n_words_from_document, self._linguistics_model.pipe(documents, batch_size=50))), index=series_index)

  def _extract_sentence_n_words_from_document(self, doc: Doc) -> List[int]:
    return list(map(self._extract_n_words_from_span, doc.sents))

  def _extract_n_words_from_span(self, span: Span) -> int:
    return sum(map(self._token_is_word, span))

  def _extract_n_words_from_document(self, doc: Doc) -> int:
    return sum(map(self._token_is_word, doc))

  def _token_is_word(self, token: Token) -> bool:
    return not any([ token.is_bracket,
                     token.is_punct,
                     token.is_currency,
                     token.is_quote,
                     token.is_space ]) # https://spacy.io/api/token#attributes

class ShorteningInference:
  def __init__ (self, checkpoint_path: Union[Path, str]):
    self._device = "cuda:0" if torch.cuda.is_available() else "cpu"

    self._model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path).to(self._device)

    self._tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

  def generate(self, sentences: List[str], batch_size: int) -> List[str]:
    sentences = list(map(str.strip, sentences))

    shortened_sentences = []

    for i in range(0, len(sentences), batch_size):
      model_inputs = self._tokenizer(sentences[i : i + batch_size], padding=True, truncation=True, return_tensors='pt').to(self._device)

      sequences = self._model.generate(**model_inputs, max_length=1000, num_beams=2)

      outputs = self._tokenizer.batch_decode(sequences, skip_special_tokens=True)

      shortened_sentences.extend(outputs)

    return shortened_sentences

class SplittingInference:
  def __init__ (self, checkpoint_path: Union[Path, str]):
    self._device = "cuda:0" if torch.cuda.is_available() else "cpu"

    self._model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path).to(self._device)

    self._tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

  def generate(self, sentences: List[str], batch_size: int) -> List[str]:
    sentences = list(map(str.strip, sentences))

    splitted_sentences = []

    for i in range(0, len(sentences), batch_size):
      model_inputs = self._tokenizer(sentences[i : i + batch_size], padding=True, truncation=True, return_tensors='pt').to(self._device)

      sequences = self._model.generate(**model_inputs, max_length=1000, num_beams=2)

      outputs = self._tokenizer.batch_decode(sequences, skip_special_tokens=True)

      splitted_sentences.extend(outputs)

    return splitted_sentences

class LongSentenceResolution:
  def __init__(self, shortening_inference: ShorteningInference, splitting_inference: SplittingInference, document_analysis: DocumentAnalysis):
    self._shortening_inference = shortening_inference

    self._splitting_inference = splitting_inference

    self._document_analysis = document_analysis
  
  def resolve(self, sentences: List[str], target_maximum_n_words: int, n_words_threshold: int, batch_size: int) -> pd.DataFrame:
    sentences = list(map(str.strip, sentences))

    sentences_df = pd.DataFrame(sentences, columns = ['input_sentence'])

    sentences_df['input_sentence_n_words'] = self._document_analysis.extract_n_words_from_documents(sentences_df['input_sentence'])

    untouched_mask = sentences_df['input_sentence_n_words'] <= target_maximum_n_words

    untouched_df = sentences_df[untouched_mask].copy()

    untouched_df['resolution'] = untouched_df['input_sentence']
    untouched_df['resolution_n_words'] = untouched_df['input_sentence_n_words']
    untouched_df['state'] = 'untouched'
    untouched_df['method'] = 'none'

    touched_df = sentences_df[~untouched_mask]

    n_words_threshold_mask = touched_df['input_sentence_n_words'] >= n_words_threshold

    splitting_df = touched_df[n_words_threshold_mask].copy()
    splitting_df['resolution'] = self._splitting_inference.generate(splitting_df['input_sentence'].values.tolist(), batch_size)
    splitting_df['resolution_n_words'] = self._document_analysis.extract_sentence_n_words_from_documents(splitting_df['resolution'])
    splitting_df['state'] = np.where(splitting_df['resolution_n_words'].apply(lambda o: all(map(lambda l: l > target_maximum_n_words, o))), "unresolved", np.where(splitting_df['resolution_n_words'].apply(lambda o: all(map(lambda l: l <= target_maximum_n_words, o))), "resolved", "partially resolved"))
    splitting_df['method'] = 'splitting'

    shortening_df = touched_df[~n_words_threshold_mask].copy()
    shortening_df['resolution'] = self._shortening_inference.generate(shortening_df['input_sentence'].values.tolist(), batch_size)
    shortening_df['resolution_n_words'] = self._document_analysis.extract_n_words_from_documents(shortening_df['resolution'])
    shortening_df['state'] = np.where(shortening_df['resolution_n_words'] > target_maximum_n_words, "unresolved", "resolved")
    shortening_df['method'] = 'shortening'

    return pd.concat([untouched_df, shortening_df, splitting_df]).sort_index()

def load_selections_for_shortening_model_dropdown (data_path: Path) -> List[str]:
  return list(map(lambda o: str(Path(os.path.basename(os.path.dirname(o))) / os.path.basename(o)), glob.glob(str(data_path / 'shortening_model_*' / 'checkpoint-*' ))))

def load_selections_for_splitting_model_dropdown (data_path: Path) -> List[str]:
  return list(map(lambda o: str(Path(os.path.basename(os.path.dirname(o))) / os.path.basename(o)), glob.glob(str(data_path / 'splitting_model_*' / 'checkpoint-*' ))))

# Connect Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

# Create paths

In [ ]:
MODELS_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Models')

# Configuration parameters

In [ ]:
long_sentence_examples = [
  "A man is facing charges after allegedly assaulting multiple people with a knife before barricading himself inside a home in Edmonton's Hazeldean neighbourhood.",
  "Tropical storm Igor is drifting northward in the Atlantic off the coast of Africa near the Cape Verde islands.",
  "Actor John Woodvine is reportedly in a stable condition after collapsing during a performance of Carousel on Friday.",
  "Greece successfully launched thursday a five-billion-euro bond issue to raise desperately needed funds, but faced stiffening opposition to its plea for help and threat to go to the imf.",
  "Singapore Airlines announced today that it has launched daily Airbus A380 service between Singapore and Hong Kong.",
  "As events do not play in favor of Europe, governments must be realistic and practical and take the decision to develop a clear and strong union.",
  "This album is by Academy Award Winning Hip Hop artist, Eminem, which was never officially released in the US, but circulated on the internet.",
  "Half of UK employees are banned from using social networking site Facebook, according to a survey from HCL Technologies.",
  "The healthcare team helped the child breathe normally, and she was back to herself quickly."
]

shortening_model_dropdown = widgets.Dropdown(options=load_selections_for_shortening_model_dropdown(MODELS_DIRECTORY_PATH), value='shortening_model_2022-10-21-21-08-16/checkpoint-61551', layout={'width': '500px'})
splitting_model_dropdown = widgets.Dropdown(options=load_selections_for_splitting_model_dropdown(MODELS_DIRECTORY_PATH), value='splitting_model_2022-10-27-15-34-55/checkpoint-135954', layout={'width': '500px'})
target_maximum_n_words_bounded_int_text = widgets.BoundedIntText(value=15, min=1, max=20, step=1)
shortening_splitting_n_words_threshold_bounded_int_text = widgets.BoundedIntText(value=20, min=1, max=100, step=1)
long_sentences_to_resolve_textarea = widgets.Textarea(placeholder='Type here long sentences to resolve.', value='\n'.join(long_sentence_examples), layout={'height': '300px', 'width': '600px'})

display(widgets.HBox([widgets.Label(value="Shortening model checkpoint:"), shortening_model_dropdown]))
display(widgets.HBox([widgets.Label(value="Splitting model checkpoint:"), splitting_model_dropdown]))
display(widgets.HBox([widgets.Label(value="Target maximum #words:"), target_maximum_n_words_bounded_int_text]))
display(widgets.HBox([widgets.Label(value="Shortening/Splitting #words threshold:"), shortening_splitting_n_words_threshold_bounded_int_text]))
display(widgets.HBox([widgets.Label(value="Long sentences to resolve:"), long_sentences_to_resolve_textarea]))

# Create dependencies

In [ ]:
document_analysis = DocumentAnalysis()

shortening_inference = ShorteningInference(MODELS_DIRECTORY_PATH / shortening_model_dropdown.value)

splitting_inference = SplittingInference(MODELS_DIRECTORY_PATH / splitting_model_dropdown.value)

long_sentence_resolution = LongSentenceResolution(shortening_inference=shortening_inference, splitting_inference=splitting_inference, document_analysis=document_analysis)

# Resolve long sentences with shortening

In [ ]:
%%time

shortening_df = pd.DataFrame({'source': long_sentences_to_resolve_textarea.value.split('\n'), 'prediction': shortening_inference.generate(sentences=long_sentences_to_resolve_textarea.value.split('\n'), batch_size=8) })

shortening_df

In [ ]:
shortening_df.to_csv('shortening-results-for-evaluation.csv', index=False)

# Resolve long sentences with splitting

In [ ]:
%%time

splitting_df = pd.DataFrame({'source': long_sentences_to_resolve_textarea.value.split('\n'), 'prediction': splitting_inference.generate(sentences=long_sentences_to_resolve_textarea.value.split('\n'), batch_size=8) })

splitting_df

In [ ]:
splitting_df.to_csv('splitting-results-for-evaluation.csv', index=False)

# Solution Recipe

In [ ]:
%%time

solution_df = long_sentence_resolution.resolve(sentences=long_sentences_to_resolve_textarea.value.split('\n'),
                                               target_maximum_n_words=target_maximum_n_words_bounded_int_text.value,
                                               n_words_threshold=shortening_splitting_n_words_threshold_bounded_int_text.value,
                                               batch_size=8)

In [ ]:
solution_df[['input_sentence', 'input_sentence_n_words', 'resolution', 'resolution_n_words', 'method', 'state']]

In [ ]:
solution_df[['input_sentence', 'input_sentence_n_words', 'resolution', 'resolution_n_words', 'method', 'state']].to_csv('solution-recipe-results-for-evaluation.csv', index=False)